In [1]:
import json
import sys
dirs = ["../..", "../create_prompt_llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
import prompt_rag

search_engine = "brave"

def load_train_bak():
    with open("covmis_data/train.json.bak", "r") as f:
        return json.load(f)
    
def load_train():
    with open("covmis_data/train.json", "r") as f:
        return json.load(f)
    
def load_type1():
    with open("output/type_1.json", "r") as f:
        return json.load(f)

def load_type2():
    with open("output/type_2.json", "r") as f:
        return json.load(f)
    
def save_type1(x):
    with open("output/type_1.json", "w") as f:
        json.dump(x, f, indent=4)
        
def save_type2(x):
    with open("output/type_2.json", "w") as f:
        json.dump(x, f, indent=4)

def item_exist(x, data):
    for item in data:
        if item["id"] == x["id"]:
            return True
    return False

def update_data_type1(x, data: list):

    data_type2 = load_type2()
    if item_exist(x, data_type2):
        return
    
    if not item_exist(x, data):
        data.append(x)

def update_data_type2(x, data: list):

    if not item_exist(x, data):
        data.append(x)

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)
try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    pass
    # data_search_llm = [{
    #     "claim": i["claim"],
    #     "claimant": i["claimant"],
    #     "label": i["label"],
    #     "date": i["date"],
    # } for i in data_search]
    # with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
    #     json.dump(data_search_llm, f, indent=4)

def save_diff(x, model_name, diff_version):
    with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
        json.dump(x, f, indent=4)

def load_diff(model_name, diff_version):
    try:
        with open(f"output/diff_{model_name}_v{diff_version}.json", "r") as f:
            return json.load(f)
    except:
        data_init = [{"claim": i["claim"], "label": int(i["label"])} for i in data_search_llm]
        with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
            json.dump(data_init, f, indent=4)
        return data_init


排查某些关键词会导致标签错误的情况

In [2]:
import nltk

# 关键词：
false_words = [
    "not", "?", "hoax", "n't", "doctored", "false", "fabricate", 
    "misinformation", "scam", "no", "in fact", "refute", "at least", "old"
]

quotation_marks = ["\"", "\'", "”", "“", "’", "‘"]
# 5 个词语以下

# 含\u201c 却没有\u201d

# \u201cCoronavirus

import pandas as pd

# data_train = load_train()
# data_type1 = load_type1()
# data_type2 = load_type2()

# for i, item in enumerate(data_train):
#     if item_exist(item, data_type1) or item_exist(item, data_type2):
#         continue

#     if item["label"] == 0:
#         label = "FALSE"
#     elif item["label"] == 1:
#         label = "PARTLY TRUE"
#     elif item["label"] == 2:
#         label = "TRUE"
#     else:
#         raise Exception()
    
#     if item["label"] == 1:
#         continue

#     claim_words = nltk.word_tokenize(item["claim"])
#     if "not" in claim_words:
#         continue

#     for word in false_words:
        
#         # 长度<=5已经核查
#         if len(claim_words) > 5 and word in nltk.word_tokenize(item["claim"]):
#             print(f'{item["id"]}({label}): {item["claim"]}')
#             break



排查句子不完整的情况

In [3]:
import os
import torch

os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'


from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
# model_type = ModelType.llama3_70b_instruct_int4

llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    # max_model_len=4096,
    # gpu_memory_utilization=0.95,
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 32,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)



2024-06-06 00:58:44,187 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-06-06 00:58:44,189 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-06-06 00:58:44,214 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 2a8987246d4b67f321effea600434525 and a total number of 976 components indexed
[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "ma

WARNING 06-06 00:58:44 config.py:1086] Casting torch.bfloat16 to torch.float16.
INFO 06-06 00:58:44 config.py:177] The model is convertible to Marlin format. Using Marlin kernel.


2024-06-06 00:58:46,672	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-06 00:58:47 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', speculative_config=None, tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=42, served_model_name=/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4)
INFO 06-06 00:58:51 utils.py:660] Found nccl from library /home/hanlv/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=639573) INFO 06-06 00:58:51 utils.py:660] Found nccl from library /home/hanlv/.config/vllm/nccl/cu1

(raylet) [2024-06-06 00:58:56,588 E 636007 636036] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-06_00-58-44_650322_635674 is over 95% full, available space: 39642611712; capacity: 1967317549056. Object creation will fail if spilling is required.


...

(raylet) [2024-06-06 00:59:16,621 E 636007 636036] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-06_00-58-44_650322_635674 is over 95% full, available space: 39642562560; capacity: 1967317549056. Object creation will fail if spilling is required.


INFO 06-06 00:59:21 distributed_gpu_executor.py:45] # GPU blocks: 2735, # CPU blocks: 4096
INFO 06-06 00:59:23 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-06 00:59:23 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(RayWorkerWrapper pid=639573) INFO 06-06 00:59:23 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(RayWorkerWrapper pid=639573) INFO 06-06 00:59:23 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per

(raylet) [2024-06-06 00:59:26,632 E 636007 636036] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-06_00-58-44_650322_635674 is over 95% full, available space: 39642554368; capacity: 1967317549056. Object creation will fail if spilling is required.


(RayWorkerWrapper pid=639573) INFO 06-06 00:59:28 model_runner.py:1017] Graph capturing finished in 5 secs.
INFO 06-06 00:59:28 model_runner.py:1017] Graph capturing finished in 6 secs.


(raylet) [2024-06-06 00:59:36,644 E 636007 636036] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-06_00-58-44_650322_635674 is over 95% full, available space: 39642546176; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-06 00:59:46,658 E 636007 636036] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-06_00-58-44_650322_635674 is over 95% full, available space: 39642497024; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-06 00:59:56,671 E 636007 636036] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-06_00-58-44_650322_635674 is over 95% full, available space: 39642206208; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-06 01:00:06,683 E 636007 636036] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-06_00-58-44_650322_635674 is over 95% full, available space: 39642206208; capacity: 1967317549056.

In [68]:
generation_config = VllmGenerationConfig(
    max_new_tokens=512,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True,
)
data_train = load_train()
prompt_list = [
    # "1 + 1?"
]
for item in data_train:
    claim = item["claim"]
    prompt = f"Please answer the following question in a short sentence! Here are 3 sentences that are incomplete due to truncation: \n\
incomplete sentences 1: la does not cause a positive COVID-19 test\n\
incomplete sentences 2: Dilma Rousseff stated\n\
incomplete sentences 3: The text claims that mass media owners, called Satanists in the text, are forcing the public to voluntarily look for a coronavirus vaccine. It also describes the vaccine (which still does not exist) as\n\
Please judge whether the following sentence is truncated like any of the above sentences. If it is truncated like the above sentences, then just answer \"Yes\", otherwise just answer \"No\":\n{claim}"
    prompt_list.append(prompt)


request_list = [{'query': prompt} for prompt in prompt_list[:100]]
resp_list = get_resp_list(request_list)
resp_list = [i["response"] for i in resp_list]

for i, resp in enumerate(resp_list):
    # print(prompt_list[i])
    print(f'{data_train[i]["id"]}: {resp.strip()}')


100%|██████████| 100/100 [00:17<00:00,  5.60it/s]

1: No. This sentence is not truncated and provides a complete thought.
2: No, this sentence is not truncated. It contains a complete thought and does not require additional information to be understood.
5: Yes, the sentence is also truncated like the incomplete sentences provided. The claim about the increase in COVID-19 mortality by 36% due to influenza vaccines is not supported by scientific evidence.
9: No, the sentence is not truncated.
11: No, this sentence is not truncated like the given examples. It is a title with two separate topics.
12: No, this sentence is not truncated like the previous examples. It contains a complete thought and does not require additional information to be understood.
13: No.
14: No, this sentence is not truncated like the given examples. It is a complete sentence describing video footage.
15: No, the sentence is complete. It conveys a clear and coherent meaning.
16: No, this sentence is not truncated like the previous examples. It is a complete sentence